##LIBRERIAS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
#drive.mount('/content/drive')

c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
 

In [2]:
#$!pip install catboost
from catboost import CatBoostClassifier
#$!pip install explainerdashboard
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
import shap
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

In [3]:
# Ruta a la carpeta en tu Google Drive
folder_path = 'Data/Cuestionario_Clima_2023_Co_Depurado (1).xlsx'
df = pd.read_excel(folder_path,header=1)

In [4]:
df.head()

,Marca temporal,Código de la institución,1. Nombre de la institución,2. Tipo de Institución,3. Denominación de los servicios instituciones en salud,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,8. Escolaridad,...,Equidad,Compromiso con la productividad,Compatibilidad de intereses,Intercambio de información,Involucración al cambio,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN,Id
0,2018-12-17 09:16:57.471,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,30,Mujer,Casado,Técnico,...,3,5,4,1,3,3.75,4.00,4.00,3.25,0
1,2018-12-21 15:26:21.193,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,27,Hombre,Soltero,Universitaria,...,3,5,5,4,4,4.75,4.75,3.50,4.50,1
2,2018-12-21 15:33:46.427,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,21,Mujer,Soltero,Técnico,...,2,5,5,4,4,4.50,4.50,3.25,4.50,2
3,2018-12-21 15:46:42.083,76001CML,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,22,Mujer,Soltero,Universitaria,...,4,5,4,3,3,4.50,4.50,3.75,3.75,3
4,2018-12-21 15:59:00.247,76001CML,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,43,Mujer,Soltero,Secundaria,...,3,5,5,4,4,4.50,4.25,3.50,4.50,4


##PREPROCESO

In [5]:
### VERIFICAMOS SI HAY FILAS CON VALORES NULOS
# Verificar si existen valores nulos en el dataframe
NullRows = df.isnull().any(axis=1).any()

# Verificar si existen cadenas vacías en el dataframe
CleanRows = (df == '').any(axis=1).any()

print(NullRows,CleanRows)

False False


In [6]:
CualitativesValues=[
#'Código de la institución  '
'1. Nombre de la institución '
,'2. Tipo de Institución '
,'3. Denominación de los servicios instituciones en salud '
,'4. Área de trabajo.'
,'5. Edad (años cumplidos)'
,'6. Sexo','7. Estado civil'
,'8. Escolaridad'
,'9. Termino de contratación'
,'10. Tipo de contrato',
'11. Horas laboradas cada día en la institución '
,'12. Tiempo Laborado en la institución'
,'13. Tiempo Laborado en su actual área o servicio'
,'14. Tiempo laborado en su actual cargo']

CualitativesValuesOneHot=[
'1. Nombre de la institución '
,'2. Tipo de Institución '
,'4. Área de trabajo.'
,'6. Sexo'
,'7. Estado civil'
,'9. Termino de contratación'
,'10. Tipo de contrato',
]
CualitativesValuesOneHotPrefix=[
'NombreInstitución '
,'TipoInstitución '
,'ÁreaTrabajo'
,'Sexo'
,'EstadoCivil'
,'TerminoContratación'
,'TipoContrato',
]

CualitativesValuesLabelEncoding=[
'3. Denominación de los servicios instituciones en salud ',
'8. Escolaridad',
'5. Edad (años cumplidos)',
'11. Horas laboradas cada día en la institución ',
'12. Tiempo Laborado en la institución',
'13. Tiempo Laborado en su actual área o servicio',
'14. Tiempo laborado en su actual cargo',
]

In [7]:
df_filter_cualidades = df.iloc[:,2:16] ##SELECCIONAMOS TODAS LAS FILAS ## ELIMINAMOS LA FECHA
df_filter_cualidades_new=pd.DataFrame(df_filter_cualidades)

In [8]:
df_filter_cualidades_new

,1. Nombre de la institución,2. Tipo de Institución,3. Denominación de los servicios instituciones en salud,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,8. Escolaridad,9. Termino de contratación,10. Tipo de contrato,11. Horas laboradas cada día en la institución,12. Tiempo Laborado en la institución,13. Tiempo Laborado en su actual área o servicio,14. Tiempo laborado en su actual cargo
0,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,30,Mujer,Casado,Técnico,Término indefinido,Prestación de servicios,1–4 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
1,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,27,Hombre,Soltero,Universitaria,Término indefinido,Por la institución,9-12 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
2,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,21,Mujer,Soltero,Técnico,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
3,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,22,Mujer,Soltero,Universitaria,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
4,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,43,Mujer,Soltero,Secundaria,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial ambulatorio,32,Hombre,Unión Libre,Técnico,Prestación de servicios,Por Cooperativa o asociación,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año
2012,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,43,Mujer,Soltero,Universitaria,Término fijo,Por la institución,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año
2013,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Apoyo operativo,29,Mujer,Soltero,Técnico,Término fijo,Por Cooperativa o asociación,5-8 horas,De 1 a 5 años,Menos de 1 año,Menos de 1 año
2014,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,37,Hombre,Unión Libre,Técnico,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años


##onehot encoding

In [9]:
# Aplica one-hot encoding a las columnas 'genero', 'tipo_contrato' y 'estado_civil'
df_encoded = pd.get_dummies(df_filter_cualidades[CualitativesValuesOneHot], prefix=CualitativesValuesOneHotPrefix)
df_encoded.columns = df_encoded.columns.str.replace('_', ' ')  # Reemplaza el separador por '.'
Arreglo = CualitativesValuesOneHot.copy()
# Concatena el DataFrame codificado con el DataFrame original
df_OneHot = pd.concat([df_filter_cualidades, df_encoded], axis=1)
df_OneHot.drop(Arreglo, axis=1, inplace=True)
df_OneHot

,3. Denominación de los servicios instituciones en salud,5. Edad (años cumplidos),8. Escolaridad,11. Horas laboradas cada día en la institución,12. Tiempo Laborado en la institución,13. Tiempo Laborado en su actual área o servicio,14. Tiempo laborado en su actual cargo,NombreInstitución CENTRO MEDICO POR SALUD,NombreInstitución CLINICA MARIANGEL TULUA,NombreInstitución CLINICA SAN JUAN DE DIOS DE MANIZALES,...,EstadoCivil Unión Libre,EstadoCivil Viudo,TerminoContratación Prestación de servicios,TerminoContratación Término fijo,TerminoContratación Término indefinido,TipoContrato Carrera administrativa,TipoContrato Otra,TipoContrato Por Cooperativa o asociación,TipoContrato Por la institución,TipoContrato Prestación de servicios
0,Baja Complejidad,30,Técnico,1–4 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,Baja Complejidad,27,Universitaria,9-12 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,Baja Complejidad,21,Técnico,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,Baja Complejidad,22,Universitaria,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,Baja Complejidad,43,Secundaria,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,Mediana complejidad,32,Técnico,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,0,0,0,...,1,0,1,0,0,0,0,1,0,0
2012,Mediana complejidad,43,Universitaria,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2013,Mediana complejidad,29,Técnico,5-8 horas,De 1 a 5 años,Menos de 1 año,Menos de 1 año,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2014,Mediana complejidad,37,Técnico,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,1,0,0,1,0,0,0,0,1,0


##ORDINAL ENCODING

In [10]:
"""
APLICAMOS LABEL ENCODING A LAS VARIABLES QUE LO AMERITAN
"""
CualitativesValuesLabelEncoding=[
'3. Denominación de los servicios instituciones en salud ',
'8. Escolaridad',
'11. Horas laboradas cada día en la institución ',
'12. Tiempo Laborado en la institución',
'13. Tiempo Laborado en su actual área o servicio',
'14. Tiempo laborado en su actual cargo',
]

df_OneHot['Denominación de los servicios instituciones en saludEncoded'] = df_OneHot['3. Denominación de los servicios instituciones en salud '].map({'Alta complejidad': 2, 'Baja Complejidad': 0, 'Mediana complejidad':1})
df_OneHot['EscolaridadEncoded'] = df_OneHot['8. Escolaridad'].map({'Primaria': 0, 'Secundaria': 1,'Técnico':2,'Universitaria':3,'Posgrado':4})
df_OneHot['HorasLaboradasEncoded'] = df_OneHot['11. Horas laboradas cada día en la institución '].map({'1–4 horas':0, '5-8 horas':1,'9-12 horas':2,'Más de 12 horas':3})
df_OneHot['TiempoInstituciónEncoded'] = df_OneHot['12. Tiempo Laborado en la institución'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot['TiempoServicioEncoded'] = df_OneHot['13. Tiempo Laborado en su actual área o servicio'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot['TiempoActualEncoded'] = df_OneHot['14. Tiempo laborado en su actual cargo'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot.drop(CualitativesValuesLabelEncoding, axis=1, inplace=True)


df_filter_cualidades_new['DenominaciónDeLosServiciosInstitucionesEnSaludEncoded'] = df_filter_cualidades['3. Denominación de los servicios instituciones en salud '].map({'Alta complejidad': 2, 'Baja Complejidad': 0, 'Mediana complejidad':1})
df_filter_cualidades_new['EscolaridadEncoded'] = df_filter_cualidades['8. Escolaridad'].map({'Primaria': 0, 'Secundaria': 1,'Técnico':2,'Universitaria':3,'Posgrado':4})
df_filter_cualidades_new['HorasLaboradasEncoded'] = df_filter_cualidades['11. Horas laboradas cada día en la institución '].map({'1–4 horas':0, '5-8 horas':1,'9-12 horas':2,'Más de 12 horas':3})
df_filter_cualidades_new['TiempoInstituciónEncoded'] = df_filter_cualidades['12. Tiempo Laborado en la institución'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_filter_cualidades_new['TiempoServicioEncoded'] = df_filter_cualidades['13. Tiempo Laborado en su actual área o servicio'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_filter_cualidades_new['TiempoActualEncoded'] = df_filter_cualidades['14. Tiempo laborado en su actual cargo'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_filter_cualidades_new.drop(CualitativesValuesLabelEncoding, axis=1, inplace=True)

In [11]:
df_filter_cualidades_new.head(5)

,1. Nombre de la institución,2. Tipo de Institución,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,9. Termino de contratación,10. Tipo de contrato,DenominaciónDeLosServiciosInstitucionesEnSaludEncoded,EscolaridadEncoded,HorasLaboradasEncoded,TiempoInstituciónEncoded,TiempoServicioEncoded,TiempoActualEncoded
0,Centro médico laboral,Privada,Asistencial ambulatorio,30,Mujer,Casado,Término indefinido,Prestación de servicios,0,2,0,1,1,1
1,Centro médico laboral,Privada,Asistencial ambulatorio,27,Hombre,Soltero,Término indefinido,Por la institución,0,3,2,1,1,1
2,Centro médico laboral,Privada,Asistencial ambulatorio,21,Mujer,Soltero,Término indefinido,Por la institución,0,2,1,1,1,1
3,Centro médico laboral,Privada,Apoyo operativo,22,Mujer,Soltero,Término indefinido,Por la institución,0,3,1,1,1,1
4,Centro médico laboral,Privada,Apoyo operativo,43,Mujer,Soltero,Término fijo,Por la institución,0,1,1,1,1,1


In [12]:
df_OneHot.head(5)

,5. Edad (años cumplidos),NombreInstitución CENTRO MEDICO POR SALUD,NombreInstitución CLINICA MARIANGEL TULUA,NombreInstitución CLINICA SAN JUAN DE DIOS DE MANIZALES,NombreInstitución CLINICA VERSALLES,NombreInstitución Centro médico laboral,NombreInstitución Christus sinergia Tuluá,NombreInstitución ESE CAMILO TRUJILLO SILVA,NombreInstitución ESE HOSPITAL LOCAL DE CANDELARIA,NombreInstitución ESE HOSPITAL MUNICIPAL ALGECIRAS,...,TipoContrato Otra,TipoContrato Por Cooperativa o asociación,TipoContrato Por la institución,TipoContrato Prestación de servicios,Denominación de los servicios instituciones en saludEncoded,EscolaridadEncoded,HorasLaboradasEncoded,TiempoInstituciónEncoded,TiempoServicioEncoded,TiempoActualEncoded
0,30,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,2,0,1,1,1
1,27,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,2,1,1,1
2,21,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,2,1,1,1,1
3,22,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,1,1,1,1
4,43,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1


In [13]:
# Obtener la lista de nombres de columnas
column_names = df_OneHot.columns.tolist()

# Cambiar el nombre de la primera columna
column_names[0] = 'Edad'

# Asignar los nuevos nombres de columna al DataFrame
df_OneHot.columns = column_names

In [14]:
df_OneHot

,Edad,NombreInstitución CENTRO MEDICO POR SALUD,NombreInstitución CLINICA MARIANGEL TULUA,NombreInstitución CLINICA SAN JUAN DE DIOS DE MANIZALES,NombreInstitución CLINICA VERSALLES,NombreInstitución Centro médico laboral,NombreInstitución Christus sinergia Tuluá,NombreInstitución ESE CAMILO TRUJILLO SILVA,NombreInstitución ESE HOSPITAL LOCAL DE CANDELARIA,NombreInstitución ESE HOSPITAL MUNICIPAL ALGECIRAS,...,TipoContrato Otra,TipoContrato Por Cooperativa o asociación,TipoContrato Por la institución,TipoContrato Prestación de servicios,Denominación de los servicios instituciones en saludEncoded,EscolaridadEncoded,HorasLaboradasEncoded,TiempoInstituciónEncoded,TiempoServicioEncoded,TiempoActualEncoded
0,30,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,2,0,1,1,1
1,27,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,2,1,1,1
2,21,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,2,1,1,1,1
3,22,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,1,1,1,1
4,43,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,32,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2,1,0,0,0
2012,43,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,3,1,0,0,0
2013,29,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2,1,1,0,0
2014,37,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,2,1,1,1,1


In [15]:
##DEFINIMOS LAS SALIDAS
df_filter_values = df.iloc[:,-5:-1]
df_filter_values

,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN
0,3.75,4.00,4.00,3.25
1,4.75,4.75,3.50,4.50
2,4.50,4.50,3.25,4.50
3,4.50,4.50,3.75,3.75
4,4.50,4.25,3.50,4.50
...,...,...,...,...
2011,3.25,3.25,3.50,3.25
2012,1.75,3.75,3.50,1.75
2013,3.50,3.75,3.75,3.25
2014,2.75,4.00,3.25,2.50


set de datos unido

In [16]:
n_bins = 5
# Asignar rangos como etiquetas
labels = range(0, n_bins)
# df_filter_values['Categoria_Liderazgo'] = pd.qcut(df_filter_values['LIDERAZGO'], q=n_bins, labels=labels)
df_filter_values['Categoria_LIDERAZGO'] = pd.qcut(df_filter_values['LIDERAZGO'], q=n_bins, labels=labels)
# Obtener los conteos de observaciones en cada bin
counts = df_filter_values['Categoria_LIDERAZGO'].value_counts().sort_index()

In [17]:
x_data=df_OneHot.iloc[:500,:]
sujeto=df_OneHot.iloc[100,:]
y_data= pd.DataFrame(df_filter_values.iloc[:500,-1], columns=['Categoria_LIDERAZGO'])

In [18]:
y_data

,Categoria_LIDERAZGO
0,3
1,4
2,4
3,4
4,4
...,...
495,2
496,2
497,2
498,0


In [19]:
type(df_filter_values)

pandas.core.frame.DataFrame

##CARGANDO MODELOS

LIDERAZGO

CLASES MANEJADAS

- GRUPO 0: (1 - 1.8)
- GRUPO 1: (1.8 - 2.6)
- GRUPO 2: (2.6 - 3.5)
- GRUPO 3: (3.5 - 4.2)
- GRUPO 4: (4.2 - 5)

cargando el modelo en xgboost

In [20]:
# Cargar el modelo desde el archivo
modelo_cargado_liderazgo = joblib.load('Modelos/LIDERAZGO_MODELO_XGB_NEW.joblib')

In [21]:
# Realizar predicciones en el conjunto de datos de validación
y_pred = modelo_cargado_liderazgo.predict(x_data)

In [22]:
y_pred

array([3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3,
       4, 4, 3, 4, 4, 4, 0, 0, 4, 0, 0, 4, 4, 4, 4, 3, 2, 4, 4, 4, 4, 0,
       4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 2, 3, 4, 2, 4, 2, 4, 2, 4, 4, 3, 4,
       3, 4, 0, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 0, 4, 3, 4, 4, 0,
       4, 0, 4, 4, 4, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 2, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 3, 0,
       4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4,
       4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4,
       0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 2, 3, 4, 4, 4,
       4, 4, 3, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 0, 4, 4, 4, 4, 3, 4, 1, 3, 4, 4, 2, 0, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 3, 4, 4, 4, 4, 4, 4, 4, 0, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       0, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 3,
       4, 3, 1, 4, 3, 4, 4, 1, 3, 4, 4, 3, 4, 0, 0,

In [23]:
# Calcular las métricas de evaluación
accuracy = accuracy_score(y_data.values, y_pred)
precision = precision_score(y_data.values, y_pred, average='macro')
recall = recall_score(y_data.values, y_pred, average='macro')
f1 = f1_score(y_data.values, y_pred, average='macro')

# Imprimir las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.922
Precision: 0.9543980717745166
Recall: 0.8493417942372521
F1-score: 0.8924411021289227


Ejecutando en explained dashboard

In [24]:
x_data

,Edad,NombreInstitución CENTRO MEDICO POR SALUD,NombreInstitución CLINICA MARIANGEL TULUA,NombreInstitución CLINICA SAN JUAN DE DIOS DE MANIZALES,NombreInstitución CLINICA VERSALLES,NombreInstitución Centro médico laboral,NombreInstitución Christus sinergia Tuluá,NombreInstitución ESE CAMILO TRUJILLO SILVA,NombreInstitución ESE HOSPITAL LOCAL DE CANDELARIA,NombreInstitución ESE HOSPITAL MUNICIPAL ALGECIRAS,...,TipoContrato Otra,TipoContrato Por Cooperativa o asociación,TipoContrato Por la institución,TipoContrato Prestación de servicios,Denominación de los servicios instituciones en saludEncoded,EscolaridadEncoded,HorasLaboradasEncoded,TiempoInstituciónEncoded,TiempoServicioEncoded,TiempoActualEncoded
0,30,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,2,0,1,1,1
1,27,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,2,1,1,1
2,21,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,2,1,1,1,1
3,22,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,1,1,1,1
4,43,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,20,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,2,3,0,0,0
496,36,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,4,3,0,0,0
497,26,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,2,2,0,0,0
498,28,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,3,2,0,0,0


In [25]:
# Crea el objeto ClassifierExplainer con el modelo entrenado
explainer = ClassifierExplainer(modelo_cargado_liderazgo, x_data, y_data.values)


Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
model_output=='probability' does not work with multiclass XGBClassifier models, so settings model_output='logodds'...
Generating self.shap_explainer = shap.TreeExplainer(model)


In [26]:
# Crea una lista con los paneles que deseas mostrar en el dashboard
panels = ['individual', 'importances', 'summary', 'contributions']

# Genera el dashboard
db = ExplainerDashboard(explainer, panels=panels,shap_interaction=True)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...


c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.


Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating shap interaction values... (this may take a while)
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.


c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:122: UserWarning:

ntree_limit is deprecated, use `iteration_range` or model slicing instead.



Generating xgboost model dump...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [43]:
db.run()

Starting ExplainerDashboard on http://192.168.1.21:8050
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app 'explainerdashboard.dashboards'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://192.168.1.21:8050
Press CTRL+C to quit
127.0.0.1 - - [29/May/2023 00:58:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 00:58:29] "GET /assets/bootstrap.min.css?m=1685309359.898529 HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2023 00:58:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 00:58:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 00:58:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2023 00:58:30] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2023 00:58:30] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2023 00:58:30] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [29/May/2023 00:58:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 00:58:30] "POST /_dash-update-compo